In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [26]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-05-08 23:00:00+00:00


In [27]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [28]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-05-08 18:03:11,456 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-08 18:03:11,472 INFO: Initializing external client
2025-05-08 18:03:11,473 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-08 18:03:12,386 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633


In [29]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.40s) 


In [30]:
df

,pickup_location_id,pickup_hour,predicted_demand
0,HB407,2025-05-08 02:00:00+00:00,0.0
1,JC072,2025-05-08 02:00:00+00:00,0.0
2,JC009,2025-05-08 02:00:00+00:00,-0.0
3,JC053,2025-05-08 02:00:00+00:00,0.0
4,HB101,2025-05-08 02:00:00+00:00,0.0
...,...,...,...
185,JC098,2025-05-08 22:00:00+00:00,1.0
186,HB603,2025-05-08 22:00:00+00:00,1.0
187,JC115,2025-05-08 22:00:00+00:00,2.0
188,HB505,2025-05-08 22:00:00+00:00,1.0


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  190 non-null    object                 
 1   pickup_hour         190 non-null    datetime64[us, Etc/UTC]
 2   predicted_demand    190 non-null    float64                
dtypes: datetime64[us, Etc/UTC](1), float64(1), object(1)
memory usage: 4.6+ KB


In [33]:
from datetime import datetime, timezone, timedelta
import pandas as pd
from src.inference import get_feature_store
import src.config as config

def fetch_next_hour_predictions():
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    start_time = pd.Timestamp(next_hour.date(), tz="UTC")
    end_time = start_time + timedelta(days=1)

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # Read predictions for the full day
    df = fg.filter(
        (fg.pickup_hour >= start_time) & 
        (fg.pickup_hour < end_time)
    ).read()

    # Filter the predictions for the exact next hour
    df_hour = df[df['pickup_hour'] == pd.Timestamp(next_hour)]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {start_time} to {end_time}")
    print(f"Found {len(df_hour)} records")

    return df_hour


# Example usage:
predictions = fetch_next_hour_predictions()

2025-05-08 18:07:02,834 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-08 18:07:02,842 INFO: Initializing external client
2025-05-08 18:07:02,842 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-08 18:07:03,865 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.41s) 
Current UTC time: 2025-05-08 22:07:02.833451+00:00
Next hour: 2025-05-08 23:00:00+00:00
Querying for date range: 2025-05-08 00:00:00+00:00 to 2025-05-09 00:00:00+00:00
Found 0 records


In [34]:
now = datetime.now(timezone.utc)

In [35]:
predictions

,pickup_location_id,pickup_hour,predicted_demand


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  190 non-null    object                 
 1   pickup_hour         190 non-null    datetime64[us, Etc/UTC]
 2   predicted_demand    190 non-null    float64                
dtypes: datetime64[us, Etc/UTC](1), float64(1), object(1)
memory usage: 4.6+ KB


In [39]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [40]:
dt = current_date.ceil('h')


In [46]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
#query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-05-08 18:12:45,248 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-08 18:12:45,255 INFO: Initializing external client
2025-05-08 18:12:45,255 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-08 18:12:46,141 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.36s) 


In [47]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.37s) 


,pickup_location_id,pickup_hour,predicted_demand


In [48]:
results

,pickup_location_id,pickup_hour,predicted_demand
0,HB407,2025-05-08 02:00:00+00:00,0.0
1,JC072,2025-05-08 02:00:00+00:00,0.0
2,JC009,2025-05-08 02:00:00+00:00,-0.0
3,JC053,2025-05-08 02:00:00+00:00,0.0
4,HB101,2025-05-08 02:00:00+00:00,0.0
...,...,...,...
185,JC098,2025-05-08 22:00:00+00:00,1.0
186,HB603,2025-05-08 22:00:00+00:00,1.0
187,JC115,2025-05-08 22:00:00+00:00,2.0
188,HB505,2025-05-08 22:00:00+00:00,1.0


In [49]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [50]:
predictions = fetch_next_hour_predictions()

2025-05-08 18:14:08,709 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-08 18:14:08,716 INFO: Initializing external client
2025-05-08 18:14:08,717 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-08 18:14:09,521 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633
Current UTC time: 2025-05-08 22:14:08.709470+00:00
Next hour: 2025-05-08 23:00:00+00:00
Found 0 records


In [51]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-05-08 18:14:10,431 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-08 18:14:10,436 INFO: Initializing external client
2025-05-08 18:14:10,437 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-08 18:14:11,319 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633


In [52]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.44s) 


In [53]:
df["pickup_hour"].max()

Timestamp('2025-05-08 22:00:00+0000', tz='Etc/UTC')

In [54]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-05-08 23:00:00+00:00


In [55]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,pickup_hour,predicted_demand
